## Getting the GSS Data

Since the data files are about 40GB zipped, we can't store a compressed or uncompressed version on GitHub, and the entire dataset can't really be loaded into memory with Colab.

One option is to use Rivana: Download the data, unzip it, and work on it in a persistent environment.

The other option is to avoid opening the entire file at once, and instead work with chunks of the data. That's what this code does for you.

On GitHub, the data are broken into three smaller files, saved in .parquet format. The code below will load these chunks into memory, one at a time, you can specify the variables you want in `var_list`, and the results will be saved in `selected_gss_data.csv`.

You can add more cleaning instructions in between the lines where the data are loaded ( `df = pd.read_parquet(url)`) and the data are saved (`df.loc...`). It's probably easiest to use this code to get only the variables you want, and then clean that subset of the data.

In [27]:
! git clone https://github.com/DS3001/project_gss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

Cloning into 'project_gss'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 23 (delta 6), reused 1 (delta 1), pack-reused 15
Receiving objects: 100% (23/23), 23.94 MiB | 20.72 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [38]:
var_list = ['wrkstat', 'prestige'] # List of variables you want to save
output_filename = 'selected_gss_data.csv' # Name of the file you want to save the data to

modes = ['w','a'] # Has write mode and append mode
phase = 0 # Starts in write mode; after one iteration of loop, switches to append mode

for k in range(3): # for each chunk of the data
    url = 'https://github.com/DS3001/project_gss/raw/main/gss_chunk_' + str(1+k) + '.parquet' # Create url to the chunk to be processed
    #print(url) # Check the url is correct
    df = pd.read_parquet(url) # Download this chunk of data
    #print(df.head()) # Visually inspect the first few rows
    df.loc[:,var_list].to_csv(output_filename, # specifies target file to save the chunk to
                              mode=modes[phase], # control write versus append
                              header=var_list, # variable names
                              index=False) # no row index saved
    phase = 1 # Switch from write mode to append mode

print(df.shape)
df.columns
# 6694 columns/variables

(24130, 6694)


Index(['year', 'id', 'wrkstat', 'hrs1', 'hrs2', 'evwork', 'occ', 'prestige',
       'wrkslf', 'wrkgovt',
       ...
       'agehef12', 'agehef13', 'agehef14', 'hompoph', 'wtssps_nea',
       'wtssnrps_nea', 'wtssps_next', 'wtssnrps_next', 'wtsscomp',
       'wtsscompnr'],
      dtype='object', length=6694)

In [103]:
# Creating new data frame with the correct variables
df2 = df[['age', 'sex', 'race', 'dipged', 'degree', 'educ', 'padeg', 'madeg',
          'major1', 'major2', 'health', 'happy',
          'marital', 'martype', 'agewed', 'divorce', 'widowed',
          'occ10', 'wrkstat', 'hrs1', 'hrs2', 'wrkslf', 'wrkgovt1',
          'wrkgovt2', 'whatslf2', 'indus10',
          'spwrksta', 'sphrs1', 'sphrs2', 'spwrkslf', 'spwrkslffam', 'spocc10',
          'sppres10']]
#df2 = df2['']
print(df2.tail())
# age (N)
# sex (C) : male or female
# race (C)
# digped (N??) : highschool education/degree
# degree (C)
# educ (N) : 10-20 ????
# paedeg/maedeg: (C) father/mother degree/education level
# major1/major2 (C) : 2 majors inputtable per person
# health (C) : poor, fair, good, excellent
# happy (C) : level
# marital (C) : status
# martype (C) : type of marriage
# agewed (N) : age married
# divorce (C) : yes/no
# widowed (C) : yes/no
# occ10 (C) : job/occ
# wrkstat : part-time, full-time, school, keeping house
# hrs1
# hrs2 (N) : typical weekly hours worked
# wrkslf (C) : self employed?
# wrkgovt1 (C) : government employeed
# wrkgovt2 (C) : private employeed
# whatslf2 (C) : work place classification
# ind10 (N) : work industry????
# ----------- ALL ABOUT SPOUSE--------------
# spwrksta
# sphrs1
# sphrs2
# spwrkslffam
# spocc10 (C) : spouse's occupation

        age     sex   race  dipged                    degree  educ  \
24125  22.0  female  white     1.0               high school  12.0   
24126  29.0  female  white     1.0                  graduate  19.0   
24127  32.0    male  white     1.0  associate/junior college  15.0   
24128  49.0  female  white     1.0                  graduate  17.0   
24129  50.0    male  white     1.0                  graduate  20.0   

             padeg                     madeg             major1     major2  \
24125  high school               high school                NaN        NaN   
24126   bachelor's                  graduate  special education  education   
24127  high school  associate/junior college             health        NaN   
24128     graduate                  graduate     home economics        NaN   
24129     graduate                  graduate            biology  chemistry   

       ... wrkgovt2                                           whatslf2  \
24125  ...      yes  other (agricult

In [ ]:
# Cleaning data to make sense in context


In [100]:
search_keywords = ['age', 'marage']
matching_columns = [col for col in df.columns if any(keyword in col for keyword in search_keywords)]

print("Matching columns:", matching_columns)

Matching columns: ['agewed', 'age', 'agekdbrn', 'aged', 'agedpar', 'agedchld', 'hitage', 'gunage', 'age1', 'age2', 'age3', 'age4', 'age5', 'switage1', 'switage2', 'newage', 'agewedcr', 'outraged', 'relagent', 'wkageism', 'agerelex', 'agerborn', 'compwage', 'agecmeus', 'setwage', 'conage', 'ageemp', 'rageemp', 'cideage', 'cideage2', 'cideage3', 'aidsage', 'aidsage2', 'aidsage3', 'suiage', 'suiage2', 'suiage3', 'intage', 'upwages', 'agedv', 'agednv', 'agehef1', 'agehef2', 'agehef3', 'agehef4', 'agehef5', 'agehef6', 'agehef7', 'agehef8', 'agehef9', 'agehef10', 'agehef11', 'agehef12', 'agehef13', 'agehef14']


In [88]:
print(df2.whatslf2)

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
24125    other (agriculture, construction, service, gov...
24126    other (agriculture, construction, service, gov...
24127    other (agriculture, construction, service, gov...
24128    other (agriculture, construction, service, gov...
24129    other (agriculture, construction, service, gov...
Name: whatslf2, Length: 24130, dtype: category
Categories (17, object): ['manufacturing', 'wholesale trade', 'retail trade',
                          'other (agriculture, construction, service, go..., ..., 'uncodeable', 'not available in this release',
                          'not available in this year', 'see codebook']


In [86]:
#for value in df['hrs2'].unique():
    #print(value)
for value, count in df['hrs2'].value_counts().items():
    print(f"{value}: {count} times")

40.0: 194 times
50.0: 50 times
45.0: 32 times
30.0: 27 times
60.0: 21 times
35.0: 17 times
15.0: 11 times
25.0: 11 times
32.0: 10 times
38.0: 9 times
20.0: 9 times
70.0: 7 times
48.0: 7 times
10.0: 7 times
55.0: 6 times
12.0: 6 times
65.0: 5 times
36.0: 5 times
5.0: 4 times
42.0: 4 times
18.0: 3 times
44.0: 3 times
24.0: 3 times
1.0: 3 times
21.0: 2 times
6.0: 2 times
8.0: 2 times
80.0: 2 times
28.0: 2 times
16.0: 2 times
46.0: 1 times
22.0: 1 times
66.0: 1 times
84.0: 1 times
34.0: 1 times
9.0: 1 times
41.0: 1 times
7.0: 1 times
3.0: 1 times
17.0: 1 times
43.0: 1 times
26.0: 1 times
52.0: 1 times
